#### Abreviaturas:
- df: data frame
- cg_coef: coeficientes de Clebsch Gordan para SU(3)
- dtot: número de degeneraciones totales

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Se importa el archivo
df = pd.read_csv('8x8comp.txt', skiprows=4, header=None)

# Se agregan las cabeceras de las columnas
df.columns = ['k', 'l', 'm', 'k1', 'l1', 'm1', 'k2', 'l2', 'm2', 'cg_coef', 'p', 'q', 'degeneracy', 'dtot']

# Se eliminan las filas con valores NaN
df = df.dropna()

# Convierte todas las columnas a tipo int con excepción de "cg_doef"
df = df.astype({"degeneracy":int, "dtot":int})
df.iloc[:,list(range(0,9))]=df.iloc[:,list(range(0,9))].astype(int)
df[['p', 'q']] = df[['p', 'q']].astype(int)

# Retorna un arreglo de arreglos de números p,q representando los multipletes existentes
multiplets = df[['p','q']].drop_duplicates().values
df

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,3,0,1,2,1,1,2,0,1,-sqrt(1/3),3,0,1,1
304,3,0,1,2,1,2,2,0,0,-sqrt(1/6),3,0,1,1
305,3,0,2,2,1,1,2,0,2,-sqrt(1/6),3,0,1,1
306,3,0,2,2,1,2,2,0,1,-sqrt(1/3),3,0,1,1


In [3]:
multiplets

array([[0, 0],
       [0, 3],
       [1, 1],
       [2, 2],
       [3, 0]])

In [4]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr

In [5]:
# Calcula el cgc para un caso específico dentro de un df previamente filtrado
def cgc_md(initial1, initial2, final, ddf):
    dff = ddf.loc[(df['k1']==initial1[0])&(df['l1']==initial1[1])&(df['m1']==initial1[2])]
    dff = dff.loc[(df['k2']==initial2[0])&(df['l2']==initial2[1])&(df['m2']==initial2[2])]
    dff = dff.loc[(df['k']==final[0])&(df['l']==final[1])&(df['m']==final[2])]

    cgc_str=dff.loc[:,'cg_coef'].values[0]
    cgc = parse_expr(cgc_str, evaluate=0)
    return cgc

# Verifica si el df filtrado es simétrico o antisimétrico
def is_symmetric(sdf):
    p1 = sdf[['k1','l1','m1']].head(1).values[0]
    p2 = sdf[['k2','l2', 'm2']].head(1).values[0]
    pf = sdf[['k', 'l', 'm']].head(1).values[0]
    val1 = cgc_md(p1, p2, pf, sdf)
    val2 = cgc_md(p2, p1, pf, sdf)
    if val1 == val2:
        return 'S'
    else:
        return 'A'

In [6]:
Symmetry = []
Multiplets = [] 
Degeneracy = []
Index = []
for i in range(len(multiplets)):
    # Retorna un df del multiplete indicado en la función
    m = df.loc[(df['p']==multiplets[i,0])&(df['q']==multiplets[i,1])]
    
    # Retorna una lista con las degeneraciones del multiplete m
    degeneracy = m['degeneracy'].value_counts().index.tolist()
    if len(degeneracy)>1:
        for j in range(len(degeneracy)):
            Multiplets.append([multiplets[i,0], multiplets[i,1]])
            
            # Divide el multiplete en df por degeneraciones
            d = m.loc[df['degeneracy']==degeneracy[j]]
            
            Index.append(d.index)
            Symmetry.append(is_symmetric(d))
            Degeneracy.append(degeneracy[j])
    else:
        Index.append(m.index)
        Multiplets.append([multiplets[i,0], multiplets[i,1]])
        Symmetry.append(is_symmetric(m))
        Degeneracy.append(degeneracy[0])

# Elabora un df para describir cada multiplete
df_describe = pd.DataFrame()
MultipletsA = np.array(Multiplets)
df_describe['p'] = MultipletsA[:,0]
df_describe['q'] = MultipletsA[:,1]
df_describe['degeneracy'] = Degeneracy
df_describe['is_symm'] = Symmetry
df_describe

,p,q,degeneracy,is_symm
0,0,0,1,S
1,0,3,1,A
2,1,1,2,S
3,1,1,1,A
4,2,2,1,S
5,3,0,1,A


In [7]:
# Agrega una columna extra al df original
extra =[]
for i in range(df.shape[0]):
    extra.append('x')
df.loc[:,'is_symm']=extra

# Asigna correctamente la característica si es simétrico o antisimétrico
for i in range(len(Index)):
    for j in Index[i]:
        df.at[Index[i],'is_symm']=df_describe.loc[i, 'is_symm']
df

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1,S
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1,S
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1,S
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1,S
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,3,0,1,2,1,1,2,0,1,-sqrt(1/3),3,0,1,1,A
304,3,0,1,2,1,2,2,0,0,-sqrt(1/6),3,0,1,1,A
305,3,0,2,2,1,1,2,0,2,-sqrt(1/6),3,0,1,1,A
306,3,0,2,2,1,2,2,0,1,-sqrt(1/3),3,0,1,1,A


### Algoritmo para el cálculo de coeficientes

#### Abrebiaciones
- I Isoespín
- Y Hipercarga
- I3 Tercera componente de Isoespín

## TO DO
1. Empezar a formar la matriz de coeficientes de decaimiento con el algoritmo
2. Dp_lambda_p = [cgc(klm, k1l1m1, k2l2m2, octete, S)]

Otra forma de buscar un valor dentro de una columna de un df es:
~~~
df[df['multiplet'].str.contains('00')]
~~~

In [8]:
e = Matrix([1,1]) # Dibarión inicial
e1 = Matrix([1,1]) # Barión final 1
e2 = Matrix([1,1]) # Barión final 2

In [9]:
dimension = (1+e[0])*(1+e[1])*(2+sum(e))/2
if e[0]<e[1]:
    print('dimension ', dimension, '*')
else:
    print('dimension ', dimension)
Imax = (e[0]+e[1])/2
print('Imax ', Imax)
Ymax = (e[0]-e[1])/3
print('Ymax ', Ymax)
Range = 2*Imax +1
print('Range ', Range)

dimension  8
Imax  1
Ymax  0
Range  3


In [10]:
# Retorna las degeneraciones de SU(2) dado un valor de I
def dec_su2(nmax):
    row = []
    
    rowImax = nmax
    size_m = 2*nmax + 1
    for i in range(size_m):
        row.insert(0,nmax)
        nmax = nmax -1
    return row

# Retorna el grupo de partículas pertenecientes al nivel solicitado
def level_particles(level):
    # Retorna la lista de la linea más larga en todo el multiplete
    I3max_line = dec_su2(Imax) # es una lista

    subIsospin3 = []
    subIsospin = []
    subHipercharge = []
    
    # Delimita los valores máximos y mínimos por nivel horizontal
    a = level-1
    b = len(I3max_line)-level + 1
    subIsospin3.append(I3max_line[a:b])
    Imax_line = I3max_line[b-1] # es un número
    list_subI = []
    list_subI.append(Imax_line)
    list_subI = list_subI*(b-a)
    subIsospin.append(list_subI)
    subImax = I3max_line[-level]
    list_subY=[]
    list_subY.append(Ymax)
    list_subY = list_subY*(b-a)
    subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia abajo formando grupos de partículas
    prow = []
    Inew = subImax
    Yrow1 = []
    Ynew1 = Ymax
    
    for i in range(e[0]-level+1):
        Inew = nsimplify(Inew -1/2)
        prow.append(Inew)
        Ynew1 = nsimplify(Ynew1 - 1)
        Yrow1.append(Ynew1)
    
    for i in range(len(prow)):
        subIsospin3.append(dec_su2(prow[i]))
        
        len_p = len(I3max_line)-i-1
        list_subI = []
        list_subI.append(prow[i])
        list_subI = list_subI*len_p
        subIsospin.append(list_subI)
        
        list_subY=[]
        list_subY.append(Yrow1[i])
        list_subY = list_subY*len_p
        subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia arriba formando grupos de partículas
    qrow = []
    Inew2 = subImax
    Yrow2 = []
    Ynew2 = Ymax
    for i in range(e[1]-level+1):
        Inew2 = nsimplify(Inew2 -1/2)
        qrow.insert(i,Inew2)
        Ynew2 = nsimplify(Ynew2 + 1)
        Yrow2.append(Ynew2)
        
    for i in range(len(qrow)):
        
        subIsospin3.insert(0,dec_su2(qrow[i]))
        list_subI = []
        list_subI.append(qrow[i])
        len_q = len(I3max_line)-i-1
        list_subI = list_subI*len_q
        subIsospin.insert(0,list_subI)
        
        list_subY=[]
        list_subY.append(Yrow2[i])
        list_subY = list_subY*len_q
        subHipercharge.insert(0, list_subY)
        
    return subHipercharge, subIsospin, subIsospin3

# Retorna el número de partículas en el grupo del nivel solicitado
def count_particles(levelp):
    tot_particles = 0
    for i in range(len(levelp)):
        tot_particles = tot_particles + len(levelp[i])
    return tot_particles

In [11]:
I3 = []
I = []
Y = []
npart = 0
for i in range(dimension):
    level = i+1
    subY, subI, subI3 = level_particles(level)
    npart = npart + count_particles(subI3)
    I3.append(subI3)
    I.append(subI)
    Y.append(subY)
    
    if npart == dimension:
        break

In [12]:
mB = []
for i in range(len(Y)):
    for j in range(len(Y[i])):
        for k in range(len(Y[i][j])):
            mB.append([Y[i][j][k], I[i][j][k], I3[i][j][k]])
print(mB)

[[1, 1/2, -1/2], [1, 1/2, 1/2], [0, 1, -1], [0, 1, 0], [0, 1, 1], [-1, 1/2, -1/2], [-1, 1/2, 1/2], [0, 0, 0]]


In [13]:
# Transforma los números cuánticos de una particula de y, i, iz al tipo k, l, m
def transform(particle):
    # Se escoge el multiplete con que se va a trabajar
    p = e[0]
    q = e[1]
    
    # Convierte enteros k, l, m
    k = int((p+2*q)/3 + particle[0]/2 + particle[1])
    l = int((p+2*q)/3 + particle[0]/2 - particle[1])
    m = int((p+2*q)/3 + particle[0]/2 + particle[2])
    return k,l,m

# Recibe una lista de lista de numeros y, i, iz la cambia a lista de enteros k,l,m
def transform_m(listm):
    list_klm = []
    for i in range(len(listm)):
        list_klm.append(transform(listm[i]))
    array_klm = np.array(list_klm)
    return array_klm

In [14]:
# Se transforman los números Y, I, I3 a k, l, m de todas las partículas de los multipletes
e0klm = transform_m(mB)
e1klm = transform_m(mB)
e2klm = transform_m(mB)

In [15]:
e0klm

array([[2, 1, 1],
       [2, 1, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1]])

In [16]:
i=1
df10=df.loc[(df['is_symm']=='S')]
df10=df10.loc[(df['p']==e[0])&(df['q']==e[1])]
df10=df10.loc[(df['k']==e0klm[i,0])&(df['l']==e0klm[i,1])&(df['m']==e0klm[i,2])]
df10

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
154,2,1,2,1,1,1,2,1,2,-sqrt(1/20),1,1,2,2,S
157,2,1,2,2,0,1,2,1,2,sqrt(3/20),1,1,2,2,S
158,2,1,2,2,0,2,2,1,1,-sqrt(3/10),1,1,2,2,S
160,2,1,2,2,1,2,1,1,1,-sqrt(1/20),1,1,2,2,S
163,2,1,2,2,1,1,2,0,2,-sqrt(3/10),1,1,2,2,S
164,2,1,2,2,1,2,2,0,1,sqrt(3/20),1,1,2,2,S


In [17]:
Gm = Matrix([])
count = 0
for i in range(len(e0klm)):
    df10=df.loc[(df['is_symm']=='S')]
    df10=df10.loc[(df['p']==e[0])&(df['q']==e[1])]
    df10=df10.loc[(df['k']==e0klm[i,0])&(df['l']==e0klm[i,1])&(df['m']==e0klm[i,2])]
    for j in range(df10.shape[0]):
        D=list(df10.iloc[j,0:3].values)
        B1=list(df10.iloc[j,3:6].values)
        B2=list(df10.iloc[j,6:9].values)
        cgci = cgc_md(B1,B2,D,df10)
        tag = [D, B1, B2]
        Gi = 'G'+str(count)
        count = count + 1
        Gm = Gm.row_insert(count, Matrix([[cgci,tag, Gi]]))

In [18]:
Gm

Matrix([
[ -sqrt(5)/10, [[2, 1, 1], [1, 1, 1], [2, 1, 1]],  G0],
[ sqrt(30)/10, [[2, 1, 1], [2, 0, 0], [2, 1, 2]],  G1],
[-sqrt(15)/10, [[2, 1, 1], [2, 0, 1], [2, 1, 1]],  G2],
[ -sqrt(5)/10, [[2, 1, 1], [2, 1, 1], [1, 1, 1]],  G3],
[-sqrt(15)/10, [[2, 1, 1], [2, 1, 1], [2, 0, 1]],  G4],
[ sqrt(30)/10, [[2, 1, 1], [2, 1, 2], [2, 0, 0]],  G5],
[ -sqrt(5)/10, [[2, 1, 2], [1, 1, 1], [2, 1, 2]],  G6],
[ sqrt(15)/10, [[2, 1, 2], [2, 0, 1], [2, 1, 2]],  G7],
[-sqrt(30)/10, [[2, 1, 2], [2, 0, 2], [2, 1, 1]],  G8],
[ -sqrt(5)/10, [[2, 1, 2], [2, 1, 2], [1, 1, 1]],  G9],
[-sqrt(30)/10, [[2, 1, 2], [2, 1, 1], [2, 0, 2]], G10],
[ sqrt(15)/10, [[2, 1, 2], [2, 1, 2], [2, 0, 1]], G11],
[-sqrt(30)/10, [[2, 0, 0], [1, 0, 0], [2, 1, 1]], G12],
[   sqrt(5)/5, [[2, 0, 0], [1, 1, 1], [2, 0, 0]], G13],
[   sqrt(5)/5, [[2, 0, 0], [2, 0, 0], [1, 1, 1]], G14],
[           0, [[2, 0, 0], [2, 0, 0], [2, 0, 1]], G15],
[           0, [[2, 0, 0], [2, 0, 1], [2, 0, 0]], G16],
[-sqrt(30)/10, [[2, 0, 0], [2, 1, 1], [

In [19]:
Gm[:,0]

Matrix([
[ -sqrt(5)/10],
[ sqrt(30)/10],
[-sqrt(15)/10],
[ -sqrt(5)/10],
[-sqrt(15)/10],
[ sqrt(30)/10],
[ -sqrt(5)/10],
[ sqrt(15)/10],
[-sqrt(30)/10],
[ -sqrt(5)/10],
[-sqrt(30)/10],
[ sqrt(15)/10],
[-sqrt(30)/10],
[   sqrt(5)/5],
[   sqrt(5)/5],
[           0],
[           0],
[-sqrt(30)/10],
[-sqrt(15)/10],
[-sqrt(15)/10],
[   sqrt(5)/5],
[   sqrt(5)/5],
[           0],
[           0],
[           0],
[-sqrt(15)/10],
[-sqrt(15)/10],
[-sqrt(30)/10],
[   sqrt(5)/5],
[   sqrt(5)/5],
[           0],
[           0],
[-sqrt(30)/10],
[ -sqrt(5)/10],
[ sqrt(15)/10],
[-sqrt(30)/10],
[ -sqrt(5)/10],
[-sqrt(30)/10],
[ sqrt(15)/10],
[ -sqrt(5)/10],
[ sqrt(30)/10],
[-sqrt(15)/10],
[ -sqrt(5)/10],
[-sqrt(15)/10],
[ sqrt(30)/10],
[  sqrt(5)/10],
[ -sqrt(5)/10],
[  -sqrt(5)/5],
[  -sqrt(5)/5],
[   sqrt(5)/5],
[  -sqrt(5)/5],
[ -sqrt(5)/10],
[  sqrt(5)/10]])

In [20]:
Gm[0:6,:]

Matrix([
[ -sqrt(5)/10, [[2, 1, 1], [1, 1, 1], [2, 1, 1]], G0],
[ sqrt(30)/10, [[2, 1, 1], [2, 0, 0], [2, 1, 2]], G1],
[-sqrt(15)/10, [[2, 1, 1], [2, 0, 1], [2, 1, 1]], G2],
[ -sqrt(5)/10, [[2, 1, 1], [2, 1, 1], [1, 1, 1]], G3],
[-sqrt(15)/10, [[2, 1, 1], [2, 1, 1], [2, 0, 1]], G4],
[ sqrt(30)/10, [[2, 1, 1], [2, 1, 2], [2, 0, 0]], G5]])

In [21]:
Gm.shape

(53, 3)

In [22]:
multiplets

array([[0, 0],
       [0, 3],
       [1, 1],
       [2, 2],
       [3, 0]])

In [23]:
transform([1, 1/2, 1/2])

(2, 1, 2)

In [24]:
def transformpq(particle, epq):
    # Se escoge el multiplete con que se va a trabajar
    p = epq[0]
    q = epq[1]
    
    # Convierte enteros k, l, m
    k = int((p+2*q)/3 + particle[0]/2 + particle[1])
    l = int((p+2*q)/3 + particle[0]/2 - particle[1])
    m = int((p+2*q)/3 + particle[0]/2 + particle[2])
    return k,l,m

In [25]:
i=0
for i in range(len(mB)):
    print(transformpq(mB[i], [1,1]))

(2, 1, 1)
(2, 1, 2)
(2, 0, 0)
(2, 0, 1)
(2, 0, 2)
(1, 0, 0)
(1, 0, 1)
(1, 1, 1)


In [26]:
# Considerando el valor de multiplets se calculan k, l, m dependiendo de p y q
def Di_pq(Dklm):
    Dpq_l=[]
    for j in range(len(multiplets)):
        Dpq_l.append(transformpq(Dklm, multiplets[j]))
    Dpq = np.array(Dpq_l)
    return Dpq

In [27]:
Di_pq(mB[0])

array([[1, 0, 0],
       [3, 2, 2],
       [2, 1, 1],
       [3, 2, 2],
       [2, 1, 1]])

In [28]:
multiplets

array([[0, 0],
       [0, 3],
       [1, 1],
       [2, 2],
       [3, 0]])

In [29]:
[0,0],[1,1],[1,1],[2,2]

([0, 0], [1, 1], [1, 1], [2, 2])

In [30]:
e0klm

array([[2, 1, 1],
       [2, 1, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1]])

In [31]:
i = 0
df11=df.loc[(df['is_symm']=='A')]
df11=df.loc[(df['p']==multiplets[i,0])&(df['q']==multiplets[i,1])]
df11=df11.loc[(df['k1']==1)&(df['l1']==1)&(df['m1']==1)]
df11

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1,S


In [32]:
i = 2
df11=df.loc[(df['is_symm']=='A')]
df11=df11.loc[(df['p']==multiplets[i,0])&(df['q']==multiplets[i,1])]
#df11=df11.loc[(df['k1']==1)&(df['l1']==1)&(df['m1']==1)]
df11.tail(40)

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
71,1,1,1,1,0,1,2,1,1,sqrt(1/4),1,1,1,2,A
72,1,1,1,1,1,1,1,1,1,sqrt(0),1,1,1,2,A
73,1,1,1,2,0,0,2,0,2,sqrt(0),1,1,1,2,A
74,1,1,1,2,0,1,2,0,1,sqrt(0),1,1,1,2,A
75,1,1,1,2,0,2,2,0,0,sqrt(0),1,1,1,2,A
76,1,1,1,2,1,1,1,0,1,-sqrt(1/4),1,1,1,2,A
77,1,1,1,2,1,2,1,0,0,sqrt(1/4),1,1,1,2,A
78,2,0,0,1,0,0,2,1,1,-sqrt(1/6),1,1,1,2,A
79,2,0,1,1,0,0,2,1,2,-sqrt(1/12),1,1,1,2,A
80,2,0,1,1,0,1,2,1,1,-sqrt(1/12),1,1,1,2,A


In [33]:
i = 2
df11=df.loc[(df['is_symm']=='S')]
df11=df11.loc[(df['p']==multiplets[i,0])&(df['q']==multiplets[i,1])]
#df11=df11.loc[(df['k1']==1)&(df['l1']==1)&(df['m1']==1)]
df11.tail(40)

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
125,1,1,1,1,0,1,2,1,1,-sqrt(1/20),1,1,2,2,S
126,1,1,1,1,1,1,1,1,1,-sqrt(1/5),1,1,2,2,S
127,1,1,1,2,0,0,2,0,2,-sqrt(1/5),1,1,2,2,S
128,1,1,1,2,0,1,2,0,1,sqrt(1/5),1,1,2,2,S
129,1,1,1,2,0,2,2,0,0,-sqrt(1/5),1,1,2,2,S
130,1,1,1,2,1,1,1,0,1,-sqrt(1/20),1,1,2,2,S
131,1,1,1,2,1,2,1,0,0,sqrt(1/20),1,1,2,2,S
132,2,0,0,1,0,0,2,1,1,-sqrt(3/10),1,1,2,2,S
133,2,0,1,1,0,0,2,1,2,-sqrt(3/20),1,1,2,2,S
134,2,0,1,1,0,1,2,1,1,-sqrt(3/20),1,1,2,2,S


In [34]:
df.dtypes

k              int64
l              int64
m              int64
k1             int64
l1             int64
m1             int64
k2             int64
l2             int64
m2             int64
cg_coef       object
p              int64
q              int64
degeneracy     int64
dtot           int64
is_symm       object
dtype: object

In [35]:
k1l1m1 = [1,1,1]
dfFiltered = df.loc[(df['is_symm']=='S')]
dfFiltered.head(15)

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1,S
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1,S
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1,S
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1,S
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1,S
5,0,0,0,2,0,2,2,0,0,sqrt(1/8),0,0,1,1,S
6,0,0,0,2,1,1,1,0,1,-sqrt(1/8),0,0,1,1,S
7,0,0,0,2,1,2,1,0,0,sqrt(1/8),0,0,1,1,S
112,1,0,0,1,0,0,1,1,1,-sqrt(1/20),1,1,2,2,S
113,1,0,1,1,0,1,1,1,1,-sqrt(1/20),1,1,2,2,S


In [36]:
dfFiltered.columns
#dfFiltered.loc[(df['k1'==1])]

Index(['k', 'l', 'm', 'k1', 'l1', 'm1', 'k2', 'l2', 'm2', 'cg_coef', 'p', 'q',
       'degeneracy', 'dtot', 'is_symm'],
      dtype='object')

In [37]:
df.head(10)

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1,S
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1,S
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1,S
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1,S
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1,S
5,0,0,0,2,0,2,2,0,0,sqrt(1/8),0,0,1,1,S
6,0,0,0,2,1,1,1,0,1,-sqrt(1/8),0,0,1,1,S
7,0,0,0,2,1,2,1,0,0,sqrt(1/8),0,0,1,1,S
9,3,0,0,1,0,0,2,0,0,-sqrt(1/2),0,3,1,1,A
10,3,0,1,1,0,0,2,0,1,-sqrt(1/3),0,3,1,1,A


In [38]:
mB[3]

[0, 1, 0]

In [39]:
# Considerando el valor de multiplets se calculan k, l, m dependiendo de p y q
def Di_pq2(Dklm,multiplete):
    return list(transformpq(Dklm, multiplete))


In [40]:
#df es una dataframe que contiene los cg__coef, klm, pq, k1l1m1 y k2m2l2
#k1l1m1 es proporcionado por el usuario (valor por defecto [1,1,1])
#e0klm = Dibariones convertidos en su versión klm.(generados previamente por otra sección de código)
#multiplete =  combinaciones necesarias de p q a procesar.
#typeSym = Para tomar decaimientos simetricos o antisimetricos.
def get_cgCoef(df,k1l1m1,e0klm,multiplets,typeSym,mB):
    cont = 0
    cgCoef_vector = []
    #Filtrado del dataframe original con unicamente datos simétricos.
    dfFiltered = df.loc[(df['is_symm']==typeSym)]
    #Filtramos unicamente con valores k1l1m1 
    dfFiltered = dfFiltered.loc[(dfFiltered['k1']== k1l1m1[0]) & (dfFiltered['l1'] == k1l1m1[1])& ( dfFiltered['m1']== k1l1m1[2])]
    #Recorremos por cada uno de los multipletes ingresados y filtramos el DF.
    for i in multiplets:
        cont=0
        dfTemporal = dfFiltered .loc[(dfFiltered ['p']==i[0]) & (dfFiltered ['q']==i[1])]
        #Recorrido con cada valor de los dibariones 
        for j in e0klm:
            dfTemporal2 = dfTemporal.loc[(dfTemporal['k2']==j[0])&(dfTemporal['l2']==j[1])&(dfTemporal['m2']==j[2])]
            #Verificamos si hubo alguna coincidencia
            if(dfTemporal2.empty):
                cgCoef_vector.append('0')
            else:
                klmConver = Di_pq2(mB[cont],i)#convertimos el valor eoklm con respecto al índice cont de e0klm
                dfTemporal3 = dfTemporal2.loc[(dfTemporal2['k']==klmConver[0])&(dfTemporal2['l']==klmConver[1])&(dfTemporal2['m']==klmConver[2])]
                if(dfTemporal3.empty):
                    continue
                else:
                   # print("i: ",i," j :",j, " klmConver: ",klmConver)
                    #Obtenemos nuestro primer valor cg_coef que será el que multipliquemos.
                    coefVal = parse_expr(dfTemporal3['cg_coef'].values[0],evaluate=0)
                    #Filtramos todos los datos cuyas coincidencias contengan a klm convertido
                    dfFinal = df.loc[(df['k']==klmConver[0])&(df['l']==klmConver[1])&(df['m']==klmConver[2])&(df['is_symm']==typeSym)]
                    for l in range(len(dfFinal)):
                        cgCoef_vector.append(Mul(parse_expr(dfFinal.cg_coef.iloc[l]),coefVal))#Multiplicamos coeficientes y encolamos.
            cont +=1
       
    return cgCoef_vector

In [41]:
multiplets

array([[0, 0],
       [0, 3],
       [1, 1],
       [2, 2],
       [3, 0]])

In [42]:
#multipletes = [[0,0],[1,1],[1,1],[2,2]]
multipletes = multiplets
k1l1m1 = [[1,1,1]]
column = []
for i in k1l1m1:
    column.append(get_cgCoef(df,i,e0klm,multipletes,'S',mB))



MatrixTest = Matrix(column)
MatrixTest

Matrix([[0, 0, 0, 0, 0, 0, 0, -1/8, 1/8, 1/8, -1/8, 1/8, -1/8, 1/8, -1/8, 0, 0, 0, 0, 0, 0, 0, 0, 1/20, -sqrt(6)/20, sqrt(3)/20, 1/20, sqrt(3)/20, -sqrt(6)/20, -3/20, -sqrt(3)/60, sqrt(6)/60, -3/20, sqrt(6)/60, -sqrt(3)/60, 1/20, -sqrt(3)/20, sqrt(6)/20, 1/20, sqrt(6)/20, -sqrt(3)/20, -3/20, -sqrt(6)/60, sqrt(3)/60, -3/20, sqrt(3)/60, -sqrt(6)/60, -sqrt(6)/10, 1/5, 1/5, 0, 0, -sqrt(6)/10, sqrt(5)/5, -sqrt(3)/10, -sqrt(3)/10, 1/5, 1/5, 0, 0, 0, -sqrt(3)/10, -sqrt(3)/10, sqrt(10)/10, sqrt(10)/10, -sqrt(6)/10, 1/5, 1/5, 0, 0, -sqrt(6)/10, sqrt(5)/5, 1/20, -sqrt(3)/20, sqrt(6)/20, 1/20, sqrt(6)/20, -sqrt(3)/20, 1/20, -sqrt(6)/20, sqrt(3)/20, 1/20, sqrt(3)/20, -sqrt(6)/20, -1/10, 1/10, 1/5, 1/5, -1/5, 1/5, 1/10, -1/10, 9/20, sqrt(6)/20, -sqrt(3)/20, 9/20, -sqrt(3)/20, sqrt(6)/20, 9/20, sqrt(3)/20, -sqrt(6)/20, 9/20, -sqrt(6)/20, sqrt(3)/20, sqrt(6)/10, 3/10, 3/10, 0, 0, sqrt(6)/10, sqrt(3)/10, sqrt(3)/10, 3/10, 3/10, 0, 0, 0, sqrt(3)/10, sqrt(3)/10, sqrt(6)/10, 3/10, 3/10, 0, 0, sqrt(6)/10,

In [43]:
MatrixTest.shape

(1, 165)